In [1]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
import random
Image.MAX_IMAGE_PIXELS = None

### 分割图片方式

#### 按照行列分割图片

In [ ]:
def image_split(img_file,split_file,raws,column):
    '''     img_file:分割图片的路径
            split_file:保存的路径
            raws:分割行数
            column:分割列数   '''
    if not os.path.exists(split_file):
        os.makedirs(split_file)
    for name in tqdm.tqdm((os.listdir(img_file))):
        image_name = name.split('.')[0]
        single_image_path = os.path.join(img_file,name)
        image = cv2.imread(single_image_path)
        #图片高度
        image_high = int(np.floor(image.shape[0]/raws))
        #图片宽度
        image_wide = int(np.floor(image.shape[1]/column))
        #创建空白图像
        cut_size = np.zeros((image_high,image_wide,3))
        #填补图像
        for i in range(raws):
            for j in range(column):
                #裁剪窗口移动
                cut_size = image[i*image_high:(i+1)*image_high,j*image_wide:(j+1)*image_wide,:]
                save_image_path = os.path.join(split_file,str(image_name)+"_"+str(i+1)+"_"+str(j+1)+"_"+".jpg")
                cv2.imwrite(save_image_path,cut_size)
i = r'D:\software\code\codefile\T\image\Root_senescence\test_sen\splic'
o = r'D:\software\code\codefile\T\image\Root_senescence\test_sen\split'
image_split(i,o,512,512)

#### 按像素分割图片

In [4]:
import cv2,os,tqdm
import numpy as np

def pixel_split_image(img_file,split_file,high_pixel,wide_pixel):
    '''     img_file:分割图片的路径
            split_file:保存的路径
            high_pixel:分割像素高度
            wide_pixel:分割像素宽度   '''
    if not os.path.exists(split_file):
        os.makedirs(split_file)
    for name in tqdm.tqdm((os.listdir(img_file))):
        image_name = name.split('.')[0]
        single_image_path = os.path.join(img_file, name)
        image = cv2.imread(single_image_path)
        H,W,C = image.shape
        #图片的行
        raws = int(np.floor(H/(high_pixel)))
        #图片的列
        column = int(np.floor(W/(wide_pixel)))

        n_H = raws*high_pixel
        n_W = column*high_pixel

        h_remainder_half = (H-n_H)//2
        w_remainder_half = (W-n_W)//2

        cut_size = np.zeros((high_pixel, wide_pixel, 3))
        for i in range(raws):
            for j in range(column):
                cut_size = image[h_remainder_half+i*(high_pixel):h_remainder_half+(i+1)*(high_pixel),w_remainder_half+j*(wide_pixel):w_remainder_half+(j+1)*(wide_pixel),:]
                save_image_path = os.path.join(split_file,str(image_name)
                    +"_"+str(i+1)+"_"+str(j+1)+"."+name.split('.')[-1])#
                cv2.imwrite(save_image_path,cut_size)

def sig_pixel_split_image(img_file,split_file,high_pixel,wide_pixel):
    '''     img_file:分割图片的路径
            split_file:保存的路径
            high_pixel:分割像素高度
            wide_pixel:分割像素宽度   '''
    if not os.path.exists(split_file):
        os.makedirs(split_file)
    for name in tqdm.tqdm((os.listdir(img_file))):
        image_name = name.split('.')[0]
        single_image_path = os.path.join(img_file, name)
        image = cv2.imread(single_image_path,cv2.IMREAD_GRAYSCALE)
        H,W = image.shape
        #图片的行
        raws = int(np.floor(H/(high_pixel)))
        #图片的列
        column = int(np.floor(W/(wide_pixel)))

        n_H = raws*high_pixel
        n_W = column*high_pixel

        h_remainder_half = (H-n_H)//2
        w_remainder_half = (W-n_W)//2

        cut_size = np.zeros((high_pixel, wide_pixel))
        for i in range(raws):
            for j in range(column):
                cut_size = image[h_remainder_half+i*(high_pixel):h_remainder_half+(i+1)*(high_pixel),w_remainder_half+j*(wide_pixel):w_remainder_half+(j+1)*(wide_pixel)]
                save_image_path = os.path.join(split_file,str(image_name)
                    +"_"+str(i+1)+"_"+str(j+1)+"."+name.split('.')[-1])#
                cv2.imwrite(save_image_path,cut_size)

i = r"D:\31890\Desktop\voc"
o = r"D:\31890\Desktop\train"
sig_pixel_split_image(i,o,512,512)


100%|██████████| 12/12 [00:10<00:00,  1.12it/s]


#### 拼接图片

In [ ]:
import cv2,os,tqdm
import numpy as np
from PIL import Image
def Stitching_images(image_file,Stitching_file,pixel):
    '''     img_file:拼接图片的路径
            Stitching_file:保存的路径  '''
    if not os.path.exists(Stitching_file):
        os.makedirs(Stitching_file)

    pixel_wide = pixel_high =pixel
    name_list = []
    for name in os.listdir(image_file):
        image_attributes = name.split('.')[-1]
        if name.split('_')[0] not in name_list:
            name_list.append(name.split('_')[0])

    for name in tqdm.tqdm(name_list):
        row_list = []
        column_list = []
        for file in os.listdir(image_file):
            if name in file:
                if file.split("_")[1] not in row_list:
                    row_list.append((file.split("_")[1]))
                if file.split("_")[2].split('.')[0] not in column_list:
                    column_list.append((file.split("_")[2].split('.')[0]))

        row = len(row_list)
        column = len(column_list)
        #创建空白画布
        target = Image.new('RGB', (pixel_wide * column, pixel_high * row))  
        #size: A 2-tuple, containing (width, height) in pixels.
        for i in range(row):
            for j in range(column):
                image = Image.open(
                    os.path.join(image_file,name+"_"+str(i+1)
                                     +"_"+str(j+1)+"."+image_attributes))
                target.paste(image, (j * pixel_wide, i * pixel_high))
        target.save(Stitching_file+"/"+str(name)+"."+image_attributes)
i = "/home/pc/T/code_file/Utils/Image/shuailao_xiao_re"
o = "/home/pc/T/code_file/Utils/Image/shuailao_re"
Stitching_images(i,o,512)

#### 多间隙分割 方法2


In [ ]:
import cv2,os,tqdm
def pixel_split_image(img_file,split_file,high_pixel,wide_pixel,overlap_pixel):
    '''     img_file:分割图片的路径
            split_file:保存的路径
            high_pixel:分割像素高度
            wide_pixel:分割像素宽度   '''
    if not os.path.exists(split_file):
        os.makedirs(split_file)
    for name in tqdm.tqdm((os.listdir(img_file))):
        image_name = name.split('.')[0]
        single_image_path = os.path.join(img_file, name)
        # print(single_image_path)
        image = cv2.imread(single_image_path)
        H,W,C = image.shape
        #图片的行
        raws = int(np.floor(H/(high_pixel-overlap_pixel)))
        #图片的列
        column = int(np.floor(W/(wide_pixel-overlap_pixel)))

        n_H = raws*high_pixel-(raws-1)*overlap_pixel
        n_W = column*high_pixel-(column-1)*overlap_pixel

        h_remainder_half = (H-n_H)//2
        w_remainder_half = (W-n_W)//2

        cut_size = np.zeros((high_pixel, wide_pixel, 3))
        for i in range(raws):
            for j in range(column):
                if i==0 and j==0:
                    cut_size = image[i*high_pixel+h_remainder_half:(i+1)*high_pixel+h_remainder_half,j*wide_pixel+w_remainder_half:(j+1)*wide_pixel+w_remainder_half,:]
                elif i==0 and j!=0:
                    cut_size = image[h_remainder_half+i*(high_pixel):h_remainder_half+(i+1)*(high_pixel), w_remainder_half+j*(wide_pixel)-(j-1)*overlap_pixel:w_remainder_half+(j+1)*(wide_pixel)-(j-1)*overlap_pixel,:]
                elif i!=0 and j==0:
                    cut_size = image[i*high_pixel-(i-1)*overlap_pixel+h_remainder_half:(i+1)*high_pixel-(i-1)*overlap_pixel+h_remainder_half,j*wide_pixel+w_remainder_half:(j+1)*wide_pixel+w_remainder_half,:]
                else:
                    cut_size = image[h_remainder_half+i*(high_pixel)-(i-1)*overlap_pixel:h_remainder_half+(i+1)*(high_pixel)-(i-1)*overlap_pixel,w_remainder_half+j*(wide_pixel)-(j-1)*overlap_pixel:w_remainder_half+(j+1)*(wide_pixel)-(j-1)*overlap_pixel,:]
                save_image_path = os.path.join(split_file,str(image_name)
                    +"_"+str(i+1)+"_"+str(j+1)+"."+name.split('.')[-1])
                # cut_size = cv2.cvtColor(cut_size,cv2.COLOR_BGR2GRAY)
                cv2.imwrite(save_image_path,cut_size)


i = r"D:\software\Code\codefile\image_result\mydata\Datasets\senescence\o_r"
o = r"D:\software\Code\codefile\image_result\mydata\Datasets\senescence\o_r_s"
pixel_split_image(i,o,768,768,24)


#### 间隙拼接   方法2

In [ ]:

def Stitching_images(image_file,Stitching_file,pixel,overlap_p):
    '''     img_file:拼接图片的路径
            Stitching_file:保存的路径  '''
    if not os.path.exists(Stitching_file):
        os.makedirs(Stitching_file)
    pixel_wide = pixel_high =pixel
    h_op=overlap_p//2
    name_list = []
    for name in os.listdir(image_file):
        if name.split('_')[0] not in name_list:
            name_list.append(name.split('_')[0])
    for name in tqdm.tqdm(name_list):
        row_list = []
        column_list = []
        for file in os.listdir(image_file):
            if name in file:
                if file.split("_")[1] not in row_list:
                    row_list.append((file.split("_")[1]))
                if file.split("_")[2].split('.')[0] not in column_list:
                    column_list.append((file.split("_")[2].split('.')[0]))
        row = len(row_list)
        column = len(column_list)
        #创建空白画布
        a_h = (row*pixel_high-(row-1)*overlap_p)
        a_w = (column*pixel_wide-(column-1)*overlap_p)
        target = np.zeros((a_h,a_w,3))
        #size: A 2-tuple, containing (width, height) in pixels.
        for i in range(row):
            for j in range(column):
                image = cv2.imread(
                    os.path.join(image_file,name+"_"+str(i+1)
                                     +"_"+str(j+1)+".png"))
                if i==0 and j==0:
                    target[i*(pixel_high-h_op):(i+1)*(pixel_high-h_op),j*(pixel_wide-h_op):(j+1)*(pixel_wide-h_op),]=image[:pixel_high-h_op,:pixel_wide-h_op,]
                elif i==0 and j!=0 and j!=column-1:
                    target[i*(pixel_high-h_op):(i+1)*(pixel_high-h_op),(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p):(pixel_high-h_op)+j*(pixel_high-overlap_p),]=image[:pixel_high-h_op,h_op:pixel_wide-h_op,]
                elif i==0 and j==column-1:
                    target[i*(pixel_high-h_op):(i+1)*(pixel_high-h_op),(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p):(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p)+(pixel_high-h_op),]=image[:pixel_high-h_op,h_op:,]
                elif i!=0 and i!=row-1 and j==0:
                    target[(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p):(pixel_high-h_op)+i*(pixel_high-overlap_p),j*(pixel_wide-h_op):(j+1)*(pixel_wide-h_op),]=image[h_op:pixel_high-h_op,:pixel_wide-h_op,]
                elif i==row-1 and j==0:
                    target[(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p):(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p)+(pixel_high-h_op),j*(pixel_wide-h_op):(j+1)*(pixel_wide-h_op),]=image[h_op:,:pixel_wide-h_op,]
                elif i==row-1 and j==column-1:
                    target[(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p):(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p)+(pixel_high-h_op),(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p):(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p)+(pixel_high-h_op),]=image[h_op:,h_op:,]
                elif i==row-1 and j!=0 and j!=column-1:
                    target[(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p):(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p)+(pixel_high-h_op),(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p):(pixel_high-h_op)+j*(pixel_high-overlap_p),]=image[h_op:,h_op:pixel_wide-h_op,]
                elif i!=row-1 and i!=0 and j==column-1:
                    target[(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p):(pixel_high-h_op)+i*(pixel_high-overlap_p),(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p):(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p)+(pixel_high-h_op),]=image[h_op:pixel_high-h_op,h_op:,]
                elif i!=row-1 and i!=0 and j!=0 and j!=column-1:
                    target[(pixel_high-h_op)+(i-1)*(pixel_high-overlap_p):(pixel_high-h_op)+i*(pixel_high-overlap_p),(pixel_high-h_op)+(j-1)*(pixel_high-overlap_p):(pixel_high-h_op)+j*(pixel_high-overlap_p),]=image[h_op:pixel_high-h_op,h_op:pixel_wide-h_op,]
        cv2.imwrite(Stitching_file+"\\"+str(name)+".png",target)
        # target.save(Stitching_file+"\\"+str(name)+".png")
i = r"D:\31890\Desktop\senescence\phenotype_compare\labels_512-68"
o = r"D:\31890\Desktop\senescence\phenotype_compare\labels2"
Stitching_images(i,o,512,68)


#### 索引图着色

In [ ]:
import os
import tqdm
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import sklearn.cluster 
import pandas as pd
#索引图制作RGB像素图
def colormap(input_file,save_file):
    '''     input_file:索引图输入图片
            save_file:RGB图片的路径   '''
    if not os.path.exists(save_file):
        os.makedirs(save_file)
    for img_name in tqdm.tqdm(os.listdir(input_file)):
        img_path = os.path.join(input_file,img_name)
        img = Image.open(img_path)
        img.convert('L')
        #调色板
        palette = [0, 0, 0,0, 128, 0, 128, 128, 0]
        # palette = [0, 0, 0, 128, 0, 0,0, 128, 0, 128, 128, 0,
        #         0, 0, 128,]
                # 128, 0, 128, 0, 128, 128, 128, 128, 128,
                # 64, 0, 0
        #着色
        img.putpalette(palette)
        img.save(os.path.join(save_file,img_name))
input_File= r'D:\31890\Pictures\image\Root_senescence\clustering_image\2\hsv\s_channel'
save_File  = r'D:\31890\Pictures\image\Root_senescence\clustering_image\2\hsv\s_channel_clustering'
colormap(input_File,save_File)

In [ ]:
import numpy as np
import cv2
import os
 
# img_h, img_w = 32, 32
img_h, img_w = 32, 48   #根据自己数据集适当调整，影响不大
means, stdevs = [], []
img_list = []
 
imgs_path = r'D:\software\Code\code-file\image\mydata\test_data\imgs\train'
imgs_path_list = os.listdir(imgs_path)
 
len_ = len(imgs_path_list)
i = 0
for item in imgs_path_list:
    img = cv2.imread(os.path.join(imgs_path,item))
    img = cv2.resize(img,(img_w,img_h))
    img = img[:, :, :, np.newaxis]
    img_list.append(img)
    i += 1
    print(i,'/',len_)    
 
imgs = np.concatenate(img_list, axis=3)
imgs = imgs.astype(np.float32) / 255.
 
for i in range(3):
    pixels = imgs[:, :, i, :].ravel()  # 拉成一行
    means.append(np.mean(pixels))
    stdevs.append(np.std(pixels))
 
# BGR --> RGB ， CV读取的需要转换，PIL读取的不用转换
means.reverse()
stdevs.reverse()

print("normMean = {}".format(means))
print("normStd = {}".format(stdevs))


#### 将图片转换为颜色图，遍历图像的方式

In [ ]:

def colormap2(image_File,original_xlsx,image_xlsx,save_image_file):
    if not os.path.exists(save_image_file):
        os.makedirs(save_image_file)

    image_pd = pd.read_excel(image_xlsx,index_col=0)
    image_columns = image_pd.columns.tolist()

    cluster_all_pd = pd.read_excel(original_xlsx,index_col=0)
    
    for column in tqdm.tqdm(image_columns):
        
        image_path = os.path.join(image_File,str(column)+'.png')
        image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
        BGR_image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image_array = BGR_image.copy()
        color_list = [[255,0,0],[255,122,0],[255,255,0],[0,255,0],[0,127,255],[0,0,255],[139,0,255],[0,255,255],[127,127,255]]#红，橙，黄，绿，青，蓝 ，紫
        for cluster in range(len(color_list)):
            temp = cluster_all_pd[cluster_all_pd[1] == cluster]
            temp_index = temp.index.tolist()

            for pixel_value in temp_index:
                for i in range(image_array.shape[0]):
                    for j in range(image_array.shape[1]):
                        if image_array[i, j, 1] == 0:
                            continue
                        if image_array[i, j, 1] == pixel_value:
                            image_array[i, j] = color_list[cluster]
        cv2.imwrite(os.path.join(save_image_file,str(column)+'.png'),image_array)

#### 其他

In [3]:
Image= r'D:\31890\Desktop\codefile\Utils\biaozhuchuli\1.png'

i = cv2.imread(Image,cv2.IMREAD_GRAYSCALE)

cv2.imwrite(r'D:\31890\Desktop\codefile\Utils\1.png',i)

True